In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.0 MB/s eta 0:00:00


In [2]:
# 1. Cài đặt thư viện
print("📦 Đang cài đặt thư viện...")
!pip install -q "transformers>=4.40.0" "pydantic<2.12"
!pip install -q datasets evaluate jiwer accelerate scikit-learn -U

import torch
from datasets import load_dataset, Image
from transformers import TrOCRProcessor, AutoTokenizer
import os

# 2. Cấu hình
MODEL_NAME = "/kaggle/input/ocr-weight"
MAX_LENGTH = 256 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Thiết bị đang dùng: {DEVICE}")

# 3. Tải và Chia Dataset
print("⏳ Đang tải dataset Neeze/CROHME-full...")
dataset = load_dataset("Neeze/CROHME-full")

# Tự động chia train/test nếu thiếu
if "test" not in dataset:
    print("⚠️ Dataset chưa có tập test. Đang chia 90/10...")
    dataset = dataset["train"].train_test_split(test_size=0.1)

# === FIX QUAN TRỌNG: TỰ ĐỘNG DÒ TÊN CỘT ===
print("🔍 Đang kiểm tra cấu trúc dataset...")
column_names = dataset["train"].column_names
print(f"   Các cột hiện có: {column_names}")

# Dò tìm tên cột ảnh (thường là 'image', 'img', hoặc cột kiểu Image)
image_col = None
text_col = None

# Ưu tiên 1: Tìm theo tên
possible_img_names = ["image", "img", "images", "picture", "pixel_values"]
possible_text_names = ["latex", "text", "label", "caption"]

for col in column_names:
    if col in possible_img_names:
        image_col = col
    if col in possible_text_names:
        text_col = col

# Ưu tiên 2: Nếu chưa tìm thấy, gán mặc định (logic fallback)
if not image_col:
    # Lấy cột đầu tiên không phải text
    image_col = [c for c in column_names if c != text_col][0]

if not text_col:
    # Lấy cột đầu tiên không phải ảnh
    text_col = [c for c in column_names if c != image_col][0]

print(f"✅ Đã xác định: Cột ảnh = '{image_col}' | Cột nhãn = '{text_col}'")

# 4. Load Processor
print("🔧 Đang load Processor...")
processor = TrOCRProcessor.from_pretrained(MODEL_NAME)
image_processor = processor.image_processor
tokenizer = processor.tokenizer

# 5. Hàm Tiền Xử Lý (On-the-fly Transform)
def transforms(examples):
    # A. Xử lý ảnh (Dùng tên cột đã dò được)
    # Lưu ý: inputs có thể là PIL Image hoặc đường dẫn file
    images = examples[image_col]
    
    # Kiểm tra nếu images là list các đường dẫn (str) thì cần load ảnh
    if len(images) > 0 and isinstance(images[0], str):
        from PIL import Image as PILImage
        images = [PILImage.open(img).convert("RGB") for img in images]
    else:
        # Nếu đã là PIL Image
        images = [img.convert("RGB") for img in images]

    pixel_values = image_processor(images, return_tensors="pt").pixel_values
    
    # B. Xử lý text (Dùng tên cột đã dò được)
    texts = examples[text_col]
    
    model_inputs = tokenizer(
        texts, 
        max_length=MAX_LENGTH, 
        truncation=True,
        padding=False 
    )
    
    model_inputs["pixel_values"] = pixel_values
    model_inputs["labels"] = model_inputs["input_ids"]
    
    return model_inputs

# 6. Áp dụng Transform
print("🚀 Đang setup pipeline (On-the-fly)...")
# Áp dụng cho từng split cụ thể để an toàn
dataset["train"].set_transform(transforms)
dataset["test"].set_transform(transforms)

# Đổi tên biến để khớp với Cell 2
processed_dataset = dataset 

print(f"Train size: {len(processed_dataset['train'])} | Test size: {len(processed_dataset['test'])}")
print("✅ CELL 1 HOÀN TẤT! Dữ liệu đã sẵn sàng.")

📦 Đang cài đặt thư viện...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
langchain-core 0.3.79 requires packaging<26.0.0,>=23.2.0, but you have packaging 26.0rc2 which is incompatible.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

2026-02-10 13:30:11.558059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770730211.750662      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770730211.808355      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770730212.261778      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770730212.261825      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770730212.261828      24 computation_placer.cc:177] computation placer alr

🚀 Thiết bị đang dùng: cuda
⏳ Đang tải dataset Neeze/CROHME-full...


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

...

Generating 2014 split:   0%|          | 0/986 [00:00<?, ? examples/s]

Generating 2016 split:   0%|          | 0/1147 [00:00<?, ? examples/s]

Generating 2019 split:   0%|          | 0/1199 [00:00<?, ? examples/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


⚠️ Dataset chưa có tập test. Đang chia 90/10...
🔍 Đang kiểm tra cấu trúc dataset...
   Các cột hiện có: ['image', 'label']
✅ Đã xác định: Cột ảnh = 'image' | Cột nhãn = 'label'
🔧 Đang load Processor...
🚀 Đang setup pipeline (On-the-fly)...
Train size: 7950 | Test size: 884
✅ CELL 1 HOÀN TẤT! Dữ liệu đã sẵn sàng.


In [3]:
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler, VisionEncoderDecoderModel
import evaluate
from tqdm.auto import tqdm
import math
import bitsandbytes as bnb
import albumentations as A
import numpy as np
from PIL import Image

# --- 1. CONFIGURATION ---
BATCH_SIZE = 24
GRADIENT_ACCUMULATION = 2
NUM_EPOCHS = 5
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.01
LABEL_SMOOTHING = 0.1
MAX_GRAD_NORM = 1.0

# --- 2. DEVICE SETUP ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 3. AUGMENTATION & TRANSFORMS ---
train_transforms = A.Compose([
    A.Rotate(limit=2, p=0.5),
    A.GaussianBlur(blur_limit=(3, 3), p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=0.2),
    A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.3),
])

def custom_transforms(examples):
    images = examples[image_col]
    pixel_values = []
    
    for image in images:
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        elif not isinstance(image, np.ndarray):
            image = image.convert("RGB")
            
        img_np = np.array(image)
        augmented = train_transforms(image=img_np)["image"]
        pixel_values.append(augmented)

    encoding = processor.image_processor(pixel_values, return_tensors="pt")
    texts = examples[text_col]
    model_inputs = tokenizer(texts, max_length=MAX_LENGTH, truncation=True, padding=False)
    
    model_inputs["pixel_values"] = encoding.pixel_values
    model_inputs["labels"] = model_inputs["input_ids"]
    return model_inputs

print("🔄 Đang cập nhật Augmentation...")
dataset["train"].set_transform(custom_transforms)

def val_transforms(examples):
    images = [img.convert("RGB") for img in examples[image_col]]
    encoding = processor.image_processor(images, return_tensors="pt")
    model_inputs = tokenizer(examples[text_col], max_length=MAX_LENGTH, truncation=True, padding=False)
    model_inputs["pixel_values"] = encoding.pixel_values
    model_inputs["labels"] = model_inputs["input_ids"]
    return model_inputs
dataset["test"].set_transform(val_transforms)

# --- 4. INIT MODEL (FIX LỖI TẠI ĐÂY) ---
print(f"🔧 Loading Model...")
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = tokenizer.sep_token_id
model.config.max_length = MAX_LENGTH
model.config.num_beams = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0

# 🔴 QUAN TRỌNG: Tắt torch.compile để tránh lỗi AttributeError 'encoder' trên Multi-GPU
# if torch.cuda.get_device_capability()[0] >= 7:
#     print("⚡ Compiling model...")
#     model = torch.compile(model) 

model.to(device)
model.train()

# Kích hoạt Multi-GPU
if torch.cuda.device_count() > 1:
    print(f"🚀 Đã kích hoạt {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

# --- 5. DATA LOADER ---
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = [item["labels"] for item in batch]
    labels_padded = tokenizer.pad({"input_ids": labels}, padding=True, return_tensors="pt").input_ids
    labels_padded[labels_padded == tokenizer.pad_token_id] = -100
    return {"pixel_values": pixel_values, "labels": labels_padded}

train_dataloader = DataLoader(
    dataset["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn, 
    num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2
)
test_dataloader = DataLoader(
    dataset["test"], batch_size=BATCH_SIZE, collate_fn=collate_fn, 
    num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2
)

# --- 6. OPTIMIZER & SCHEDULER ---
optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

num_training_steps = NUM_EPOCHS * len(train_dataloader) // GRADIENT_ACCUMULATION
lr_scheduler = get_scheduler(
    name="cosine", optimizer=optimizer, 
    num_warmup_steps=int(0.1 * num_training_steps), num_training_steps=num_training_steps
)

scaler = torch.cuda.amp.GradScaler() # Dùng cú pháp cũ ổn định hơn cho T4

# --- 7. TRAINING LOOP ---
print("🚀 Bắt đầu Training...")
cer_metric = evaluate.load("cer")
best_cer = float("inf")
progress_bar = tqdm(range(num_training_steps))

# Lấy config (Xử lý trường hợp DataParallel)
if hasattr(model, "module"):
    model_config = model.module.config
else:
    model_config = model.config

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    optimizer.zero_grad()
    
    for step, batch in enumerate(train_dataloader):
        for k, v in batch.items():
            batch[k] = v.to(device)

        with torch.cuda.amp.autocast(): # Dùng autocast cuda thay vì generic amp
            outputs = model(**batch)
            logits = outputs.logits
            
            loss_fct = CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING, ignore_index=-100)
            loss = loss_fct(logits.view(-1, model_config.vocab_size), batch["labels"].view(-1))
            loss = loss / GRADIENT_ACCUMULATION

        scaler.scale(loss).backward()

        if (step + 1) % GRADIENT_ACCUMULATION == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
            optimizer.zero_grad()
            
            progress_bar.update(1)
            progress_bar.set_description(f"Ep {epoch+1} | Loss: {loss.item()*GRADIENT_ACCUMULATION:.4f}")

        total_train_loss += loss.item() * GRADIENT_ACCUMULATION

    avg_train_loss = total_train_loss / len(train_dataloader)
    
    # --- EVALUATION ---
    print(f"\n📊 Đánh giá Epoch {epoch+1}...")
    
    # Unwrap model để eval
    unwrapped_model = model.module if hasattr(model, "module") else model
    unwrapped_model.eval()
    
    valid_cer = 0.0
    valid_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Eval"):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)
            
            generated_ids = unwrapped_model.generate(pixel_values)
            pred_str = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            labels[labels == -100] = tokenizer.pad_token_id
            label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            cer = cer_metric.compute(predictions=pred_str, references=label_str)
            valid_cer += cer
            valid_batches += 1
            
    avg_cer = valid_cer / valid_batches
    print(f"✅ Epoch {epoch+1} | Loss: {avg_train_loss:.4f} | CER: {avg_cer:.4f}")

    if avg_cer < best_cer:
        best_cer = avg_cer
        print(f"💾 Kỷ lục mới (CER: {best_cer:.4f}). Đang lưu model...")
        save_path = "/kaggle/working/math_ocr_best_optimized"
        unwrapped_model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        processor.save_pretrained(save_path)

print("\n🎉 HOÀN TẤT HUẤN LUYỆN!")

/tmp/ipykernel_24/4289966388.py:29: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
/tmp/ipykernel_24/4289966388.py:30: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=0.2),


🔄 Đang cập nhật Augmentation...
🔧 Loading Model...
🚀 Đã kích hoạt 2 GPUs!
🚀 Bắt đầu Training...


/tmp/ipykernel_24/4289966388.py:121: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Dùng cú pháp cũ ổn định hơn cho T4


  0%|          | 0/830 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_24/4289966388.py:144: FutureWarning: `torch.cuda.amp.autocast(args...

...

✅ Epoch 3 | Loss: 1.4572 | CER: 0.1173
💾 Kỷ lục mới (CER: 0.1173). Đang lưu model...

📊 Đánh giá Epoch 4...


Eval:   0%|          | 0/37 [00:00<?, ?it/s]

✅ Epoch 4 | Loss: 1.4334 | CER: 0.1144
💾 Kỷ lục mới (CER: 0.1144). Đang lưu model...

📊 Đánh giá Epoch 5...


Eval:   0%|          | 0/37 [00:00<?, ?it/s]

✅ Epoch 5 | Loss: 1.4255 | CER: 0.1083
💾 Kỷ lục mới (CER: 0.1083). Đang lưu model...

🎉 HOÀN TẤT HUẤN LUYỆN!
